In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_absolute_error, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import matplotlib.pyplot as plt
import pickle
import hyperopt.pyll.stochastic
import time
from xgboost import XGBRegressor

/home/phe002/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test['loss'] = np.nan
joined = pd.concat([train, test])

In [3]:
features = [x for x in train.columns if x not in ['id','loss', 'log_loss']]

cat_features = [x for x in train.select_dtypes(
        include=['object']).columns if x not in ['id','loss', 'log_loss']]
num_features = [x for x in train.select_dtypes(
        exclude=['object']).columns if x not in ['id','loss', 'log_loss']]

print "Categorical features:", len(cat_features)
print "Numerical features:", len(num_features)

Categorical features: 116
Numerical features: 14


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


In [5]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels)),False

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat)),False

def mae_score(y_true, y_pred):
    return mean_absolute_error(np.exp(y_true), np.exp(y_pred))

mae_scorer = make_scorer(mae_score, greater_is_better=False)

In [6]:
df = pd.DataFrame({'A':[np.nan,'type1','type2'],
                   'B':['type1','type2','type3'],
                   'C':['type1','type3','type3']})
print pd.factorize(df['A'].values, sort=True)[0]
print pd.factorize(df['A'].values)[0]
print pd.factorize(df['B'].values, sort=True)[0]
print pd.factorize(df['C'].values, sort=True)[0]

[-1  0  1]
[-1  0  1]
[0 1 2]
[0 1 1]


In [6]:
for column in list(train.select_dtypes(include=['object']).columns):
    if train[column].nunique() != test[column].nunique():
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train

        remove = (remove_train|remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x

        joined[column] = joined[column].apply(lambda x: 'np.nan' if x in remove else x, 1)

    joined[column] = pd.factorize(joined[column].values, sort=True)[0]

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 200
y = np.log(train['loss'] + shift)
ids = test['id']
X = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)
lgtrain = lgb.Dataset(X, label=y,categorical_feature=cat_features,free_raw_data=False)
lgtest = lgb.Dataset(X_test)

In [10]:
#1148.9639894 {'reg_alpha': 0.13614510960026047, 'colsample_bytree': 0.48613283826428166, 'scale_pos_weight': 1, 'learning_rate': 0.018415349849039475, 'nthread': 10, 'min_child_weight': 3, 'subsample': 0.8045758877474857, 'max_depth': 16, 'gamma': 3.738381824865164}
RANDOM_STATE = 0
'''params = {
    'min_child_weight': 1,
    'eta': 0.01,
    'colsample_bytree': 0.5,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 1,
    'gamma': 1,
    'silent': 1,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'nthread':10
}'''
params={'colsample_bytree': 0.6, 'learning_rate': 0.01, 
             'min_child_weight': 6, #'n_estimators': 100, 
            'nthread': 10,
            'subsample': 0.9, 'objective': 'regression', 
             'max_depth': 9, 'min_split_gain': 0.4,'reg_alpha': 0.0001,'seed':0}
model = lgb.train(params, lgtrain, 10, feval=evalerror)

/home/phe002/anaconda2/lib/python2.7/site-packages/lightgbm/basic.py:1002: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [11]:
eta_list  = [0.1]*100 + [0.05]*200 + [0.02]*400
#model = xgb.train(params, xgtrain, 700, feval=evalerror)

In [8]:
lgb_params={'colsample_bytree': 0.6, 'learning_rate': 0.01, 
            'min_child_weight': 6,
            'nthread': 5, 
            'subsample': 0.9, 'objective':'regression',
             'max_depth': 9, 'min_split_gain': 0.4,'reg_alpha': 0.0001,'seed':0}

In [9]:
lgb_params={ 'nthread': 10, 
            'num_leaves':120,'min_data_in_leaf':100,'max_depth':9}

In [10]:
#%%time
cv_result_lgb = lgb.cv(lgb_params, lgtrain, num_boost_round=10, nfold=5, seed=0, stratified=False,
                    feval=evalerror, early_stopping_rounds=None)
print cv_result_lgb['mae-mean'][-1]
print('\nBest num_boost_round:', len(cv_result_lgb['mae-mean']))

1412.44216116
('\nBest num_boost_round:', 10)


In [10]:
print cv_result_lgb['mae-mean'][-1]
print('\nBest num_boost_round:', len(cv_result_lgb['mae-mean']))

1192.46133299
('\nBest num_boost_round:', 3000)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)

ax1.set_title('100 rounds of training')
ax1.set_xlabel('Rounds')
ax1.set_ylabel('Loss')
ax1.grid(True)
ax1.plot(bst_cv1[['train-mae-mean', 'test-mae-mean']])
ax1.legend(['Training Loss', 'Test Loss'])

ax2.set_title('60 last rounds of training')
ax2.set_xlabel('Rounds')
ax2.set_ylabel('Loss')
ax2.grid(True)
ax2.plot(bst_cv1.iloc[1750:][['train-mae-mean', 'test-mae-mean']])
ax2.legend(['Training Loss', 'Test Loss'])
fig.set_size_inches(16,4)
#fig.subplots_adjust(hspace=0)
plt.show()

In [107]:
from sklearn.model_selection import StratifiedKFold
Xt = np.zeros((1000,100))
t = np.ones((1000,100))
yt=np.concatenate((np.zeros(500),np.ones(500)))
skf = StratifiedKFold(n_splits=2)
for train_index, test_index in skf.split(Xt, yt):
    t[test_index] = 0
np.where(t)

(array([], dtype=int64), array([], dtype=int64))

In [44]:
def plottrend(grid=grid,bins=bins):    
    test_scores_mean = grid.cv_results_['mean_test_score']
    test_scores_std = grid.cv_results_['std_test_score']
    plt.grid()
    plt.fill_between(bins, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,color="r")
    plt.plot(bins, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.show()

In [ ]:
%%time
bins = range(10,230,1)
lgb_params = {
    'min_data_in_leaf': bins,
    'num_leaves':[100],
     'max_depth':[9],
    'num_threads':[10],
}
clf=lgb.LGBMRegressor()
grid = GridSearchCV(clf,param_grid=lgb_params, cv=3, scoring=mae_scorer)
grid.fit(X, y)

print grid.best_score_
print grid.best_params_

In [ ]:
plottrend(grid=grid,bins=bins)

In [134]:
%%time
# The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
xgb_params = {
    'learning_rate': [0.1],
    'n_estimators':[100],
     'gamma':[ 0.1 * i for i in range(0,5)],
    'colsample_bytree': [0.5],
    'subsample': [0.5],
    'objective': ['reg:linear'],
    'max_depth': [9],
    'min_child_weight': [6],
    'nthread':[10],
    'num_boost_round':[150]
}
clf=XGBoostRegressor()
grid = GridSearchCV(clf,param_grid=xgb_params, cv=3, scoring=mae_scorer)
grid.fit(X, y)

print grid.best_score_
print grid.best_params_

-1156.26686109
{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'nthread': 10, 'min_child_weight': 6, 'n_estimators': 100, 'subsample': 0.5, 'objective': 'reg:linear', 'num_boost_round': 150, 'max_depth': 9, 'gamma': 0.4}
CPU times: user 43min 4s, sys: 23.3 s, total: 43min 27s
Wall time: 4min 30s


In [135]:
%%time
# The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
xgb_params = {
    'learning_rate': [0.1],
    'n_estimators':[100],
     'gamma':[0.4],
    'subsample':[i/10.0 for i in range(1,10)],
     'colsample_bytree':[i/10.0 for i in range(1,10)],
    'objective': ['reg:linear'],
    'max_depth': [9],
    'min_child_weight': [6],
    'nthread':[10],
    'num_boost_round':[150]
}
clf=XGBoostRegressor()
grid = GridSearchCV(clf,param_grid=xgb_params, cv=3, scoring=mae_scorer)
grid.fit(X, y)

print grid.best_score_
print grid.best_params_

-1150.47117641
{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'nthread': 10, 'min_child_weight': 6, 'n_estimators': 100, 'subsample': 0.9, 'objective': 'reg:linear', 'num_boost_round': 150, 'max_depth': 9, 'gamma': 0.4}
CPU times: user 9h 6min 51s, sys: 4min 47s, total: 9h 11min 39s
Wall time: 57min 2s


In [136]:
%%time
# The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
xgb_params = {
    'learning_rate': [0.5,0.4,0.3,0.2,0.1,0.075,0.05,0.04,0.03],
    'n_estimators':[500],
     'gamma':[0.4],
    'subsample':[0.9],
     'colsample_bytree':[0.6],
    'objective': ['reg:linear'],
    'max_depth': [9],
    'min_child_weight': [6],
    'nthread':[10],
    'num_boost_round':[150]
}
clf=XGBoostRegressor()
grid = GridSearchCV(clf,param_grid=xgb_params, cv=3, scoring=mae_scorer)
grid.fit(X, y)

print grid.best_score_
print grid.best_params_

-1150.47117641
{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'nthread': 10, 'min_child_weight': 6, 'n_estimators': 500, 'subsample': 0.9, 'objective': 'reg:linear', 'num_boost_round': 150, 'max_depth': 9, 'gamma': 0.4}
CPU times: user 1h 11min 42s, sys: 37 s, total: 1h 12min 19s
Wall time: 7min 28s


In [142]:
%%time
# The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
xgb_params = {
    'learning_rate': [0.1],
    'reg_alpha':[1e-5,1e-4,1e-3, 1e-2, 0.1, 1,10,100],
    'n_estimators':[100],
     'gamma':[0.4],
    'subsample':[0.9],
     'colsample_bytree':[0.6],
    'objective': ['reg:linear'],
    'max_depth': [9],
    'min_child_weight': [6],
    'nthread':[10],
    'num_boost_round':[150]
}
clf=XGBoostRegressor()
grid = GridSearchCV(clf,param_grid=xgb_params, cv=3, scoring=mae_scorer)
grid.fit(X, y)

print grid.best_score_
print grid.best_params_

-1149.84640685
{'reg_alpha': 0.0001, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 'nthread': 10, 'min_child_weight': 6, 'n_estimators': 100, 'subsample': 0.9, 'objective': 'reg:linear', 'num_boost_round': 150, 'max_depth': 9, 'gamma': 0.4}
CPU times: user 1h 5min 45s, sys: 34.6 s, total: 1h 6min 19s
Wall time: 6min 53s


In [34]:
prediction = np.exp(model.predict(xgtest)) - shift
submission = pd.DataFrame()
submission['loss'] = prediction
submission['id'] = ids
submission.to_csv('sub_v.csv', index=False)

In [ ]:
if __name__ == '__main__':
    print "a+b"